In [1]:
import os
backend = 'pytorch'
os.environ['DGLBACKEND'] = backend

In [2]:
# import torch
import tensorflow as tf
import dgl
import networkx as nx
import tqdm.auto as tqdm
import numpy as np
import itertools
import pickle
from matplotlib import pyplot as plt

Using backend: pytorch


# Parse solutions

In [3]:
G = nx.Graph()

with open('../data/tsp10_concorde_2000.txt') as data_file:
    for line in data_file:
        problem_str, solution_str = line.split('output')

        node_data = iter(problem_str.strip().split(' '))
        node_counter = 0
        nodes = {}
        for x in node_data:
            y = next(node_data)
            pos = np.array([float(x), float(y)])
            nodes[node_counter] = pos
            node_counter += 1

        solution = [int(x) - 1 for x in solution_str.strip().split(' ')]
        solution_edges = [e for e in zip(solution[:-1], solution[1:])]

        offset = len(G.nodes)
        for i in nodes:
            G.add_node(offset + i, x=nodes[i][0], y=nodes[i][1])
        for i, j in itertools.combinations(nodes, 2):
            w = np.linalg.norm(nodes[i] - nodes[j])
            in_solution = (i, j) in solution_edges or (j, i) in solution_edges
            G.add_edge(offset + i, offset + j, in_solution=in_solution, weight=w)

# Compute features

In [4]:
def greedy_tour(g, depot, weight='weight'):
    tour = [depot]
    while len(tour) < len(g.nodes):
        i = tour[-1]
        neighbours = [(j, g.edges[(i, j)]['weight']) for j in g.neighbors(i) if j not in tour]
        j, dist = min(neighbours, key=lambda e: e[1])
        tour.append(j)

    tour.append(depot)
    return tour

def set_greedy_tour(g, depot):
    tour = greedy_tour(g, depot)
    tour_edges = zip(tour[:-1], tour[1:])

    nx.set_edge_attributes(g, False, 'in_greedy_solution')
    for e in tour_edges:
        g.edges[e]['in_greedy_solution'] = True
        

In [5]:
def set_neighbour_features(g, nn_levels, min_degree):
    # calculate knn for each edge
    for e in g.edges:
        g.edges[e]['neighbour'] = {}

    for i in g.nodes:
        neighbours = [(j, g.edges[(i, j)]['weight']) for j in g.neighbors(i)]
        nearest_neighbours = sorted(neighbours, key=lambda e: e[1])
        for k, (j, _) in enumerate(nearest_neighbours):
            g.edges[(i, j)]['neighbour'][i] = k
            
    # knn graphs and nn clique
    nx.set_edge_attributes(g, False, 'nn_clique')
    for i, level in enumerate(nn_levels):
        nx.set_edge_attributes(g, False, f'{i}_nn')

    for e in g.edges:
        i, j = e
        neighbours = g.edges[e]['neighbour']
        if neighbours[i] == neighbours[j]:
            g.edges[e]['nn_clique'] = True

        for level_i, level in enumerate(nn_levels):
            g.edges[(i, j)][f'{level_i}_nn'] = (neighbours[i] <= level) or (neighbours[j] <= level)
            
    # erode longest edges until min degree reached
    edges = sorted([(e, g.edges[e]['weight']) for e in g.edges], key=lambda e: e[1], reverse=True)
    edges, _ = map(list, zip(*edges))

    h = g.edge_subgraph(edges)
    while min(dict(nx.degree(h)).values()) > min_degree:
        edges.pop(0)
        h = g.edge_subgraph(edges)

    nx.set_edge_attributes(g, False, 'md_nn')
    for e in edges:
        g.edges[e]['md_nn'] = True


In [6]:
def set_depot_weight(g, depot):
    for n in g.nodes:
        if n == depot:
            g.nodes[n]['depot_weight'] = 0
        else:
            g.nodes[n]['depot_weight'] = g.edges[(depot, n)]['weight']
            

In [7]:
def set_fancy_graph_features(g):
    cf_bc = nx.edge_current_flow_betweenness_centrality(g, weight='weight')
    sp_bc = nx.edge_betweenness_centrality(g, weight='weight')

    sp_cc = nx.closeness_centrality(g, distance='weight')
    cf_cc = nx.current_flow_closeness_centrality(g, weight='weight')
    cl = nx.clustering(g, weight='weight')

    nx.set_edge_attributes(g, sp_bc, 'sp_betweenness')
    nx.set_edge_attributes(g, cf_bc, 'cf_betweenness')
    nx.set_node_attributes(g, sp_cc, 'sp_closeness')
    nx.set_node_attributes(g, cf_cc, 'cf_closeness')
    nx.set_node_attributes(g, cl, 'clustering')
    

# Prepare dataset

In [8]:
for nodes in tqdm.tqdm(nx.connected_components(G)):
    g = G.subgraph(nodes)

    depot = next(iter(g.nodes))
    set_greedy_tour(g, depot)
    set_depot_weight(g, depot)

    nn_levels = [int(x*len(g.nodes)) for x in [0.1, 0.2, 0.3]]
    min_degree = 2
    set_neighbour_features(g, nn_levels, min_degree)

    set_fancy_graph_features(g)
    
    for e in g.edges:
        i, j = e
        features = np.array([
            g.edges[e]['weight'],
            g.edges[e]['in_greedy_solution'],
            g.edges[e]['neighbour'][i],
            g.edges[e]['neighbour'][j],
            g.edges[e]['nn_clique'],
            g.edges[e]['0_nn'],
            g.edges[e]['1_nn'],
            g.edges[e]['2_nn'],
            g.edges[e]['md_nn'],
#             g.edges[e]['sp_betweenness'], # the same for every node
            g.edges[e]['cf_betweenness'],
            g.nodes[i]['depot_weight'],
            g.nodes[j]['depot_weight'],
            g.nodes[i]['sp_closeness'],
            g.nodes[j]['sp_closeness'],
            g.nodes[i]['cf_closeness'],
            g.nodes[j]['cf_closeness'],
            g.nodes[i]['clustering'],
            g.nodes[j]['clustering'],
        ], dtype=np.float32)
        label = np.array([
            g.edges[e]['in_solution'],
        ]).astype(np.int64)
        
        g.edges[e]['x'] = features
        g.edges[e]['y'] = label

|          | 0/? [00:00<?, ?it/s]

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(np.vstack([g.edges[e]['x'] for e in g.edges]))

In [10]:
dgl_graphs = []
nx_graphs = []

for nodes in tqdm.tqdm(nx.connected_components(G)):
    g = G.subgraph(nodes)
    lg = nx.line_graph(g)
    lg.add_edges_from(zip(lg.nodes, lg.nodes))
    
    for e_i, e in enumerate(lg.nodes):
        features = g.edges[e]['x']
        label = g.edges[e]['y']
        lg.nodes[e]['x'] = scaler.transform(features[np.newaxis, :]).squeeze()
        lg.nodes[e]['y'] = label
        lg.nodes[e]['e'] = e
        
    dgl_g = dgl.from_networkx(lg, node_attrs=['x', 'y', 'e'])
    
    dgl_graphs.append(dgl_g)
    nx_graphs.append(g)
    

|          | 0/? [00:00<?, ?it/s]

In [11]:
dgl.data.utils.save_graphs(f'../data/dgl_graphs_{backend}.bin', dgl_graphs)

In [12]:
nx.write_gpickle(nx_graphs, f'../data/nx_graphs_{backend}.pkl')